In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
path = os.path.join('/content/drive/MyDrive/')

Mounted at /content/drive


##**Read data files** ##

In [ ]:
#!mkdir /content/drive/MyDrive/ViLT_/photos
#!cp -r /content/drive/MyDrive/ViLT/photos/* /content/drive/MyDrive/ViLT_/photos

In [ ]:
import pandas as pd
import os

path = '/content/drive/MyDrive/ViLT'
train_file = os.path.join(path, 'combined_train.csv')
val_file = os.path.join(path, 'combined_val.csv')
test_file = os.path.join(path, 'combined_test.csv')

train_df = pd.read_csv(train_file)
val_df = pd.read_csv(val_file)
test_df = pd.read_csv(test_file)
print(train_df.head(1))

   Unnamed: 0      ID     Lat      Lon  Average  Variance      Votes  \
0       86447  115344  52.672 -3.03426      2.4      0.24  2,3,2,3,2   

                              Geograph URI  gridimage_id      imageclass  ...  \
0  http://www.geograph.org.uk/photo/582085        582085  Farm buildings  ...   

   imagetaken user_id moderation_status average tags_word_count  \
0  2007-10-10    4582          geograph     2.4               1   

  comments_word_count              clean_comments scenic  \
0                   7  northern end long mountain      0   

                                           input_ids  \
0  [101, 2642, 2203, 2146, 3137, 102, 0, 0, 0, 0,...   

                                      attention_mask  
0  [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  

[1 rows x 25 columns]


**Download photos : no need to run again !!!**

In [ ]:
import os
from PIL import Image

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, urlparse, urlunparse, urlretrieve
import os
import sys

# download image using url and save to the local storage
def download_dataframe_img(dataframe):
  img_count = 0
  for i, (index, row) in enumerate(dataframe.iterrows()):
    # prepare dirs for image
    img_path = os.path.join(path, row['filename'])
    os.makedirs(os.path.split(img_path)[0], exist_ok=True)
    img_name = os.path.split(img_path)[1]
    img_url = row['Geograph URI']

    # download image from url
    soup = bs(urlopen(img_url))
    parsed = list(urlparse(img_url))


    for image in soup.findAll("img"):
      filename = image["src"].split("/")[-1]
      if filename==img_name:
        # print('Downloading image from : %s'%img_url)
        img_count +=1
        if (img_count % 100)==0:
          print('downloaded %d images'%img_count)
        parsed[2] = image["src"]
        outpath = os.path.join(os.path.split(img_path)[0], filename)
        os.makedirs(os.path.split(outpath)[0], exist_ok=True)
        if image["src"].lower().startswith("http"):
            urlretrieve(image["src"], outpath)

        break

download_dataframe_img(train_df[:])
download_dataframe_img(val_df[:])
download_dataframe_img(test_df[:])

Downloaded !!!

Read train, validation, test data

##**Finetune the ViLT model** ##

In [ ]:
# !pip install lightning
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.5 MB/s eta 0:00:00


Load pretrained ViLT model

In [ ]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import torch

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
# print(model)

In [ ]:
# test code
from PIL import Image
from matplotlib.pyplot import imshow
import torch
from torchvision.io import read_image
import numpy as np

# img = Image.open('/content/drive/MyDrive/ViLT/sample.jpg')
img = np.asarray(read_image('/content/drive/MyDrive/ViLT/sample.jpg'))
comment = 'scenic view'
# print(np.asarray(img))

In [ ]:
# test code
encoding = processor([img], [comment], return_tensors='pt', padding='max_length')
for value in encoding.values():
  print(value.size())
print(encoding.keys())
embeds = model.vilt(**encoding)['pooler_output']
# print((img))
print(embeds.size())

In [ ]:
import os
import shutil

for path in train_df['filename']:
  if not os.path.exists(os.path.join('/content/drive/MyDrive/ViLT_', path)):
    shutil.copy(os.path.join('/content/drive/MyDrive/ViLT', path), os.path.join('/content/drive/MyDrive/ViLT_', path))
    print(path)

for path in test_df['filename']:
  if not os.path.exists(os.path.join('/content/drive/MyDrive/ViLT_', path)):
    shutil.copy(os.path.join('/content/drive/MyDrive/ViLT', path), os.path.join('/content/drive/MyDrive/ViLT_', path))
    print(path)

for path in val_df['filename']:
  if not os.path.exists(os.path.join('/content/drive/MyDrive/ViLT_', path)):
    shutil.copy(os.path.join('/content/drive/MyDrive/ViLT', path), os.path.join('/content/drive/MyDrive/ViLT_', path))
    print(path)


crate dataloaders

In [ ]:
# old version : do not run

from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torchvision.transforms import Resize
import numpy as np

class ViltDataset(Dataset):
  def __init__(self, dataframes):
    self.base_path = '/content/drive/MyDrive/ViLT_'
    self.resize = Resize((384, 512))
    self.img_paths = dataframes['filename']
    self.images = []
    for img_path in self.img_paths:
      img = self.resize(read_image(os.path.join(self.base_path, img_path), mode=torchvision.io.ImageReadMode.RGB))
      self.images.append(np.asarray(img))

    self.comments = dataframes['clean_comments']
    self.labels = dataframes['scenic']

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    image = self.images[idx]
    comment = self.comments[idx]
    encoding = processor(image, comment, return_tensors='pt', padding='max_length', truncation=True)
    for key in encoding.keys():
      encoding[key] = torch.squeeze(encoding[key], 0)
    label = torch.nn.functional.one_hot(torch.tensor(self.labels[idx]), num_classes=2)

    return [encoding, label]

new dataset

In [ ]:
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torchvision.transforms import Resize
import torchvision
import numpy as np

class ViltDataset(Dataset):
  def __init__(self, dataframes):
    self.base_path = '/content/drive/MyDrive/ViLT'
    self.resize = Resize((384, 512))
    # self.img_paths = dataframes['filename']
    # self.images = []
    self.encodings = []
    img_paths = dataframes['filename']
    comments = dataframes['clean_comments']

    for i in range(len(img_paths)):
      img = self.resize(read_image(os.path.join(self.base_path, img_paths[i]), mode=torchvision.io.ImageReadMode.RGB))
      # print(img.size())
      comment = comments[i]
      encoding = processor(img, comment, return_tensors='pt', padding='max_length', truncation=True)
      for key in encoding.keys():
        encoding[key] = torch.squeeze(encoding[key], 0)
      self.encodings.append(encoding)

    self.labels = dataframes['scenic']

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    encoding = self.encodings[idx]
    label = torch.nn.functional.one_hot(torch.tensor(self.labels[idx]), num_classes=2)

    return [encoding, label]

In [ ]:
# create dataloaders
# training_data = ViltDataset(train_df[:2000])
test_data = ViltDataset(test_df[:200])
val_data = ViltDataset(val_df[:200])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
train_dataloader = DataLoader(training_data, batch_size=40, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=20, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=20, shuffle=True)

In [ ]:
# test code
encoding_, label_ = next(iter(val_dataloader))
model.to('cpu')
for key in encoding_.keys():
  print(key)
  print(encoding_[key].size())
# embeds = model.vilt(**encoding)['pooler_output']
# print(embeds)

input_ids
torch.Size([4, 40])
token_type_ids
torch.Size([4, 40])
attention_mask
torch.Size([4, 40])
pixel_values
torch.Size([4, 3, 384, 512])
pixel_mask
torch.Size([4, 384, 512])


Define a classifier model

In [ ]:
class ScenicClassifier(torch.nn.Module):
  def __init__(self, enc_features: int):
    super().__init__()
    self.embedding = model.vilt
    self.classifier = torch.nn.Sequential(
        torch.nn.Linear(enc_features, 32, bias=False),
        torch.nn.ReLU(),
        torch.nn.Linear(32, 2, bias=False),
        # torch.nn.ReLU()
        )

  def forward(self, encodings):
    embeds = self.embedding(**encodings)['pooler_output']
    logits = self.classifier(embeds)

    return logits

In [ ]:
class ScenicClassifier2(torch.nn.Module):
  def __init__(self, enc_features: int):
    super().__init__()
    self.embedding = model.vilt
    self.classifier = torch.nn.Linear(enc_features, 2, bias=False)
    torch.nn.init.xavier_uniform(self.classifier.weight)

  def forward(self, encodings):
    embeds = self.embedding(**encodings)['pooler_output']
    # softmax = torch.nn.Softmax(dim=1)
    # logits = softmax(self.classifier(embeds))
    logits = self.classifier(embeds)

    return logits

In [ ]:
device='cuda:0'
# classifier = ScenicClassifier(768)
classifier = ScenicClassifier2(768)
classifier.to(device)

criterion = torch.nn.MSELoss()
criterion.to(device)
# optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
# optimizer = torch.optim.SGD(classifier.classifier.parameters(), lr=0.005, momentum=0.8)
optimizer = torch.optim.SGD(classifier.parameters(), lr=0.001, momentum=0.8)

<ipython-input-23-f2ddaa6a3100>:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.classifier.weight)


In [ ]:
# test code
print(classifier.parameters())
for parameter in classifier.classifier.parameters():
  print(parameter)
  print(parameter.size())

## Train the model

In [ ]:
classifier.train()
for epoch in range(4):
  running_loss = 0.0
  for i, data in enumerate(train_dataloader, 0):
    encodings, labels = data
    encodings.to(device)
    labels.to(device)
    # for value in encodings.values():
    #   print(value.size())
    # print(labels)
    # break

    optimizer.zero_grad()
    outputs = classifier(encodings)
    labels = torch.tensor(labels, dtype=torch.float).cuda()
    # labels.to(device)
    # print(outputs)
    # print(labels)
    # print('################')
    # for parameter in classifier.classifier.parameters():
    #   print(parameter)
    # print('################')
    loss = criterion(outputs, labels)
    # print(loss)
    # break
    loss.backward()
    optimizer.step()
    # for parameter in classifier.classifier.parameters():
    #   print(parameter)
    # print('################')
    # break
    running_loss += loss.item()
    if i % 10 == 9:
      print(f'[{epoch+1}, {i+1:5d}] loss : {running_loss / 10:.3f}')
      # for parameter in classifier.classifier.parameters():
      #   print(parameter)
      running_loss=0.0

print('Finished training')

<ipython-input-26-7a095b61de1e>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float).cuda()


[1,    10] loss : 0.675
[1,    20] loss : 0.249
[1,    30] loss : 0.151
[1,    40] loss : 0.124
[1,    50] loss : 0.123
[2,    10] loss : 0.103
[2,    20] loss : 0.096
[2,    30] loss : 0.088
[2,    40] loss : 0.091
[2,    50] loss : 0.091
[3,    10] loss : 0.085
[3,    20] loss : 0.082
[3,    30] loss : 0.075
[3,    40] loss : 0.075
[3,    50] loss : 0.067
[4,    10] loss : 0.066
[4,    20] loss : 0.066
[4,    30] loss : 0.071
[4,    40] loss : 0.063
[4,    50] loss : 0.068
Finished training


In [ ]:
!del optimizer

torch.cuda.empty_cache()
!nvidia-smi -caa
!nvidia-smi


/bin/bash: line 1: del: command not found
Cleared Accounted PIDs for GPU 00000000:00:04.0.
All done.
Wed Aug 23 05:35:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    35W /  70W |   2249MiB / 15360MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-----------

## Test the result

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for testdata in val_dataloader:
    encodings, labels = testdata
    encodings = encodings.to(device)
    output = classifier(encodings)
    _, predicted = torch.max(output.data, 1)
    total+= labels.size(0)
    # print(predicted)
    # print(labels)
    # print(torch.argmax(labels, 1))
    # break
    correct+=(predicted.to('cpu')==torch.argmax(labels.to('cpu'), 1)).sum().item()

print(f'Accuracy of the network : {100*correct//total}%')

Accuracy of the network : 93%




### TRASH BELOW ###



In [ ]:
# text_embeds = model.vilt.embeddings.text_embeddings(encoding['input_ids'])
# print(encoding['input_ids'])
# encoding = encoding['input_ids', 'token_type_ids', 'pixel_values', 'pixel_mask']

# output = model(**encoding)
# print(output)
# text_emb = model.vilt.embeddings.text_embeddings(**encoding)
# print(output.keys())

In [ ]:
# img_embedding = processor.image_processor(image)
# print(img_embedding)
# encoding = processor(image, text, return_tensors='pt')
# print(encoding.keys())